In [23]:
suppressMessages(suppressWarnings(library(dplyr)))
# Generate some example data to work with - TODO: move this into it's own script
sales_person <- rep("Abbie", 5)
sales_person <- append(sales_person, rep("Brenda", 5))
sales_person <- append(sales_person, rep("Carl", 5))
sales_person <- append(sales_person, rep("Dirk", 5))
sales_person <- append(sales_person, rep("Ed", 5))
region <- rep('west', 5)                  # Abbie is in west region
region <- append(region, rep('east', 5))  # Brenda is in east region
region <- append(region, rep('east', 5))  # Carl is in east region
region <- append(region, rep('west', 5))  # Dirk is in west region
region <- append(region, rep('east', 5))  # Ed is in east region
sale_date <- rep(seq(as.Date("2018-01-01"), by = "day", length.out = 5), 5)
forecast_a <- 1.3 * seq(1, 5, by = 1)
forecast_a <- append(forecast_a, 1.7 * seq(1, 5, by = 1))
forecast_a <- append(forecast_a, 0.9 * seq(1, 5, by = 1))
forecast_a <- append(forecast_a, 2.1 * seq(1, 5, by = 1))
forecast_a <- round(append(forecast_a, 0.5 * seq(1, 5, by = 1)), 2)
forecast_b <- round(0.8 * forecast_a, 2)
forecast_c <- round(0.8 * forecast_b, 2)
# add a little noise to the forecast to simulate actual looking values
set.seed(201801)
actual_a <- round(forecast_a + rnorm(25, 0, 0.2), 2)
actual_b <- round(forecast_b + rnorm(25, 0, 0.2), 2)
actual_c <- round(forecast_c + rnorm(25, 0, 0.2), 2)

sales_data <- tibble(sale_date, sales_person, region,
                     forecast_a, forecast_b, forecast_c,
                     actual_a, actual_b, actual_c)
head(sales_data, 12) #[c(1,5,6,10,11,15,16,20,21,25),]

sale_date,sales_person,region,forecast_a,forecast_b,forecast_c,actual_a,actual_b,actual_c
2018-01-01,Abbie,west,1.3,1.04,0.83,1.38,1.08,0.72
2018-01-02,Abbie,west,2.6,2.08,1.66,2.55,1.94,2.03
2018-01-03,Abbie,west,3.9,3.12,2.50,4.05,2.82,2.63
2018-01-04,Abbie,west,5.2,4.16,3.33,4.86,4.12,3.47
2018-01-05,Abbie,west,6.5,5.20,4.16,6.09,4.98,4.09
2018-01-01,Brenda,east,1.7,1.36,1.09,1.72,1.41,0.94
2018-01-02,Brenda,east,3.4,2.72,2.18,3.16,2.97,2.03
2018-01-03,Brenda,east,5.1,4.08,3.26,5.06,3.75,3.18
2018-01-04,Brenda,east,6.8,5.44,4.35,6.46,5.51,4.76
2018-01-05,Brenda,east,8.5,6.80,5.44,8.67,6.86,5.64


In [27]:
sales_data %>% group_by(region) %>% summarize(sum_forecast_a = sum(forecast_a))
sales_data %>% group_by(region) %>% summarize(sum_forecast_b = sum(forecast_b))
sales_data %>% group_by(region) %>% summarize(sum_forecast_c = sum(forecast_c))

region,sum_forecast_a
east,46.5
west,51.0


region,sum_forecast_b
east,37.2
west,40.8


region,sum_forecast_c
east,29.76
west,32.64


In [34]:
sumByVar1 <- function(df, sum_col) {
    quo_sum_col <- enquo(sum_col)
    ret_df <- df %>% group_by(region) %>% summarize(sum_forecast = sum(!!quo_sum_col))
    return(ret_df)
}

sumByVar1(sales_data, forecast_a)

region,sum_forecast
east,46.5
west,51.0


In [37]:
sumByVar2 <- function(df, grp_col, sum_col) {
    quo_grp_col <- enquo(grp_col)
    quo_sum_col <- enquo(sum_col)
    ret_df <- df %>% group_by(!!quo_grp_col) %>% summarize(sum_forecast = sum(!!quo_sum_col))
    return(ret_df)
}

sumByVar2(sales_data, region, forecast_b)

region,sum_forecast
east,37.2
west,40.8


In [44]:
sumByVar3 <- function(df, grp_col, sum_col) {
    quo_grp_col <- enquo(grp_col)
    quo_sum_col <- enquo(sum_col)
    sum_name <- paste0("sum_", quo_name(quo_sum_col))
    ret_df <- df %>% group_by(!!quo_grp_col) %>% summarize(!!sum_name := sum(!!quo_sum_col))
    return(ret_df)
}

sumByVar3(sales_data, region, forecast_c)

region,sum_forecast_c
east,29.76
west,32.64


## Refences

1.  Programming with dplyr - Hadley Wickham [https://rpubs.com/hadley/dplyr-programming](https://rpubs.com/hadley/dplyr-programming)